In [1]:
# First I uploaded the images as zipped file and now I will unzip it by 
import zipfile
with zipfile.ZipFile('images/images.zip', 'r') as zip_ref:
    zip_ref.extractall('images/')

it created jpg folder,  it unzipped the file and create the images inside jpg folder 

In [3]:
# we will divide the images in categories and create folder per category
# the category will be like category name-category id
# the categories are stored in categories.txt file 
import os
import csv
import shutil

categories_file_path = 'categories.txt'
# read category names from categories.txt file
categories = open(categories_file_path, "r")
category_names = categories.read().split('\n')
category_paths = []

for i in range(len(category_names)):
    name = category_names[i] + "-" + str(i + 1)
    category_path=os.path.join('dataset', name)
    category_paths.append(category_path)
    if not os.path.exists(category_path):
        os.makedirs(category_path)

images_info=[]
with open('imagelabels.csv', mode='r') as read_csv:
    for image in read_csv:
        images_info.append(image.split())

# copy the images to the new folder that we created depend on each image category        
images_path = 'images/jpg'

for image in os.listdir(images_path):
    image_name = image.split('.')[0]
    image_number = image_name.split('_')[1]
    image_num = int(image_number)
    category_id = images_info[image_num - 1]
    image_path = category_paths[int(category_id[0])-1]
    current_path=os.path.join(images_path,image)
    new_path=os.path.join(image_path,image)
    if not os.path.exists(new_path):
        shutil.copyfile(current_path,new_path)

In [12]:
# to make sure that all the images transfered successfully we will print the number of the category folders and the number of the images

images = 0
categories = []
for i in os.listdir('dataset'):
    categories.append(i)
    
for n in categories:
    for m in os.listdir('dataset/' + n):
        images += 1
print("the number of the images is " + str(images))    
print("the number of the categories is " + str(len(categories)))

the number of the images is 8189
the number of the categories is 102


In [6]:
# I will use GrabCut to segment the foreground from the bac ground of the image 
# GrabCut is an image segmentation method based on graph cuts.
# the refrence of the algorithm that i will create on geeks for geeks website on 
# https://www.geeksforgeeks.org/python-foreground-extraction-in-an-image-using-grabcut-algorithm/


# Python program to illustrate  
# foreground extraction using 
# GrabCut algorithm 
   
# organize imports 
import numpy as np 
import cv2 
from matplotlib import pyplot as plt 
   
def image_segmentation(image_name):    
    # path to input image specified and  
    # image is loaded with imread command 
    image = cv2.imread(image_name) 

    # create a simple mask image similar 
    # to the loaded image, with the  
    # shape and return type 
    mask = np.zeros(image.shape[:2], np.uint8) 

    # specify the background and foreground model 
    # using numpy the array is constructed of 1 row 
    # and 65 columns, and all array elements are 0 
    # Data type for the array is np.float64 (default) 
    backgroundModel = np.zeros((1, 65), np.float64) 
    foregroundModel = np.zeros((1, 65), np.float64) 

    # define the Region of Interest (ROI) 
    # as the coordinates of the rectangle 
    # where the values are entered as 
    # (startingPoint_x, startingPoint_y, width, height) 
    # these coordinates are according to the input image 
    # it may vary for different images 
    height, width, _ = image.shape
    # now we specify the rectangle of ROI
    # we will use threshold as crop the image to be 12% in (top, down, left,right)
    # I think it will be better 
    ROI_threshold=0.12
    rectangle = (int(ROI_threshold * height) , int(ROI_threshold * width), 
                 int(height-(ROI_threshold * height)), int(width-(ROI_threshold * width))) 

    # apply the grabcut algorithm with appropriate 
    # values as parameters, number of iterations = 3  
    # cv2.GC_INIT_WITH_RECT is used because 
    # of the rectangle mode is used  
    cv2.grabCut(image, mask, rectangle,   
                backgroundModel, foregroundModel, 
                3, cv2.GC_INIT_WITH_RECT) 

    # In the new mask image, pixels will  
    # be marked with four flags  
    # four flags denote the background / foreground  
    # mask is changed, all the 0 and 2 pixels  
    # are converted to the background 
    # mask is changed, all the 1 and 3 pixels 
    # are now the part of the foreground 
    # the return type is also mentioned, 
    # this gives us the final mask 
    mask2 = np.where((mask == 2)|(mask == 0), 0, 1).astype('uint8') 

    # The final mask is multiplied with  
    # the input image to give the segmented image. 
    image = image * mask2[:, :, np.newaxis]
    
    return image

In [7]:
# now we will go to the training data step but before that , 
# we need to segment all images and copy the segmented images into a new seperated folder to use them in the training 
import os

segmented_dataset_folder='segmentation_data'

# for each sub folder in dataset create a folder with same name but contains the segmented images
for f in os.listdir('dataset'):
    category_path=os.path.join('dataset',f)
    segmented_folder=os.path.join(segmented_dataset_folder,f)
    if not os.path.exists(segmented_folder):
        os.makedirs(segmented_folder)
    for img in os.listdir(category_path):
        new_image_path=os.path.join(segmented_folder,img)
        if not os.path.exists(new_image_path):
            img_path=os.path.join(category_path,img)
            segmented_image=image_segmentation(img_path)
            cv2.imwrite(new_image_path,segmented_image)

In [30]:
# first we will isolate the test images in two folders , one for the segmentation and one for non segmentation images
# we will take 20% of the dataset for testing
# first we will start with non segmentation test

import os
import shutil
import random

non_seg_test_folder="non_segmentation_test"
if not os.path.exists(non_seg_test_folder):
    os.makedirs(non_seg_test_folder)
        
for folder in os.listdir('dataset'):
    category_path=os.path.join('dataset',folder)
    test_folder_path=os.path.join(non_seg_test_folder,folder)
    if not os.path.exists(test_folder_path):
        os.makedirs(test_folder_path)
    num_images=len(os.listdir(category_path))
    number_of_tests=int((num_images/100)*20)
    category_names=os.listdir(category_path)
    list_randoms=[]
    images_already_moved=False
    if len(os.listdir(test_folder_path))>0:
        images_already_moved=True
    if not images_already_moved:
        while True:
            ran=random.randint(1,number_of_tests)
            if ran not in list_randoms:
                list_randoms.append(ran)
            if len(list_randoms)==number_of_tests:
                break
                
        for n in list_randoms:
            image_path=os.path.join(category_path,category_names[n-1])
            new_image_path=os.path.join(test_folder_path,category_names[n-1])
            if not os.path.exists(new_image_path):
                shutil.move(image_path,new_image_path)

In [31]:
# now we will start with the segmentation test
# we will take 20% of the dataset for testing
# first we will start with segmentation test

import os
import shutil
import random

seg_test_folder="segmentation_test"
if not os.path.exists(seg_test_folder):
    os.makedirs(seg_test_folder)
        
for folder in os.listdir('dataset'):
    category_path=os.path.join('dataset',folder)
    test_folder_path=os.path.join(seg_test_folder,folder)
    if not os.path.exists(test_folder_path):
        os.makedirs(test_folder_path)
    num_images=len(os.listdir(category_path))
    number_of_tests=int((num_images/100)*20)
    category_names=os.listdir(category_path)
    list_randoms=[]
    images_already_moved=False
    if len(os.listdir(test_folder_path))>0:
        images_already_moved=True
    if not images_already_moved:
        while True:
            ran=random.randint(1,number_of_tests)
            if ran not in list_randoms:
                list_randoms.append(ran)
            if len(list_randoms)==number_of_tests:
                break
                
        for n in list_randoms:
            image_path=os.path.join(category_path,category_names[n-1])
            new_image_path=os.path.join(test_folder_path,category_names[n-1])
            if not os.path.exists(new_image_path):
                shutil.move(image_path,new_image_path)

In [59]:
# I read a lot before starting and this is the link I got to read from 
# https://www.tensorflow.org/hub/tutorials/image_retraining
# create folders and paramaters that we will need for our training
from random import shuffle
import os.path
import numpy as np
import tensorflow as tf
from tensorflow.python.framework import graph_util
from tensorflow.keras import Model, layers
from tensorflow.python.platform import gfile

lr = 0.02
training_steps = 10000
display_step = 10
num_classes=102

test_image_dir_with_segmentation="segmentation_test/"
test_image_dir_without_segmentation="non_segmentation_test/"
train_image_dir_with_segmentation="segmentation_data/"
train_image_dir_without_segmentation="dataset/"

output_graph_path_without_segmentation="model/retrained_graph_without_segmentation/"
if not os.path.exists(output_graph_path_without_segmentation):
    os.makedirs(output_graph_path_without_segmentation)
    
output_graph_path_with_segmentation="model/retrained_graph_with_segmentation/"
if not os.path.exists(output_graph_path_with_segmentation):
    os.makedirs(output_graph_path_with_segmentation)

inception_model_dir="inceptionV3_model/"
if not os.path.exists(inception_model_dir):
    os.makedirs(inception_model_dir)

train_bottleneck_dir_with_segmentation="./tf_files/train_bottlenecks_with_segmentation/"
if not os.path.exists(train_bottleneck_dir_with_segmentation):
    os.makedirs(train_bottleneck_dir_with_segmentation)

train_bottleneck_dir_without_segmentation="./tf_files/train_bottlenecks_without_segmentation/"
if not os.path.exists(train_bottleneck_dir_without_segmentation):
    os.makedirs(train_bottleneck_dir_without_segmentation)

# it's a better size after a while testing
BOTTLENECK_TENSOR_SIZE = 2048

BOTTLENECK_TENSOR_NAME = 'pool_3/_reshape:0'
JPEG_DATA_TENSOR_NAME = 'DecodeJpeg/contents:0'
RESIZED_INPUT_TENSOR_NAME = 'ResizeBilinear:0'
final_tensor_name = 'final_result'

In [60]:
# we start learning with original images with no segmentation then with segmented images and compare the results
# now we will create a function that takes the category name and return the label 
def read_label(category_name):
    arr=category_name.split('_')
    category_name=arr[1]
    label =  np.zeros(num_classes, dtype=np.float32)
    file_read = open("categories_names.txt", "r")
    labels  = file_read.read().split('\n')
    for i in range(len(labels)):
        if category_name == labels[i]:
            label[i] = 1.0
            return label

In [61]:
# we create this function to read bottlnecks contains the resulted 2048 folat number of the image
def load_bottlenecks(Bottleneck_dir):
    Bottlenecks = []
    labels=[]
    counter = 0
    index = 0 
    for Class in os.listdir(Bottleneck_dir):
        class_path = os.path.join(Bottleneck_dir, Class)
        for item in os.listdir(class_path):
            bottleneck_path = os.path.join ( class_path, item )
            with open ( bottleneck_path, 'r' ) as bottleneck_file:
                bottleneck_string = bottleneck_file.read ()
            bottleneck_values = [float ( x ) for x in bottleneck_string.split ( ',' )]
            label = get_label ( Class )
            Bottlenecks.append (bottleneck_values  )
            labels.append(label)
            counter += 1
        index += 1

    return Bottlenecks, labels

In [62]:
# create inception graph function
# refrence : https://www.programcreek.com/python/example/90293/tensorflow.GraphDef

def create_inception_graph():
    with tf.Graph().as_default() as graph:
        model_filename = os.path.join(inception_model_dir, 'classify_image_graph_def.pb')
    with gfile.FastGFile(model_filename, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())

    bottleneck_tensor, jpeg_data_tensor, resized_input_tensor = (
          tf.import_graph_def(graph_def, name='', return_elements=[
              BOTTLENECK_TENSOR_NAME, JPEG_DATA_TENSOR_NAME,
              RESIZED_INPUT_TENSOR_NAME]))
    return graph, bottleneck_tensor, jpeg_data_tensor, resized_input_tensor

In [36]:
def Create_Image_lists(image_path):
    # load the images for train and store into result dictionary 
    result = {}
    for img in os.listdir(image_path):
        training_images = []
        for file in os.listdir(os.path.join(image_path,img)):
            training_images.append(file)
            
        #save the label and the names to the dictionary
        result[img] = {'dir': img, 'training': training_images}
    return result    

In [37]:
# get the file path
def get_path(image_lists, label_name, index, image_dir, category):

    if label_name not in image_lists:
        tf.logging.fatal('Label does not exist %s.', label_name)
    label_lists = image_lists[label_name]
    if category not in label_lists:
        tf.logging.fatal('Category does not exist %s.', category)
    category_list = label_lists[category]
    if not category_list:
        tf.logging.fatal('Label %s has no images in the category %s.', label_name, category)
    mod_index = index % len(category_list)
    base_name = category_list[mod_index]
    sub_dir = label_lists['dir']
    full_path = os.path.join(image_dir, sub_dir, base_name)
    print(full_path)
    return full_path

In [77]:
def create_bottleneck_file(bottleneck_path, image_lists, label_name, index,
                           image_dir, category, sess, jpeg_data_tensor,bottleneck_tensor):
    
    image_path = get_path(image_lists, label_name, index, image_dir, category)
    if not gfile.Exists(image_path):
        tf.logging.fatal('File does not exist %s', image_path)

    image_data = gfile.FastGFile(image_path, 'rb').read()
    try:
        print("create bottle ", sess.__dict__)
        bottleneck_values = sess.run(bottleneck_tensor, {jpeg_data_tensor: image_data})
        bottleneck_values = np.squeeze(bottleneck_values)
        bottleneck_string = ','.join(str(x) for x in bottleneck_values)
        
        with open(bottleneck_path, 'w') as bottleneck_file:
            bottleneck_file.write(bottleneck_string)
        print("bottleneck file created successfully")    

    except Exception as e:
        print('Error during processing file %s' % image_path)
        print(e)

    

In [91]:
def cache_bottlenecks(sess, image_lists, image_dir, bottleneck_dir, jpeg_data_tensor, bottleneck_tensor):

    if not os.path.exists(bottleneck_dir):
        os.makedirs(bottleneck_dir)
    bottlenecks = 0
    for label_name, label_lists in image_lists.items ():
        for category in ['training']:

            # get images names in each category
            category_list = label_lists[category]

            for index, unused_base_name in enumerate(category_list):
                # if the bottleneck is already exist load it ,if not create it from the graph
                label_lists = image_lists[label_name]
                sub_dir = label_lists['dir']
                sub_dir_path = os.path.join(bottleneck_dir, sub_dir)
                if not os.path.exists(sub_dir_path):
                    os.makedirs(sub_dir_path)

              # create bottleneck path for each image
                bottleneck_path = get_path(image_lists, label_name, index, bottleneck_dir, category)
                bottleneck_path += '.txt' 
                if not os.path.exists(bottleneck_path):
                    create_bottleneck_file(bottleneck_path, image_lists, label_name, index, image_dir, category, sess, jpeg_data_tensor, bottleneck_tensor)

                with open(bottleneck_path, 'r') as bottleneck_file:
                    bottleneck_string = bottleneck_file.read()
                hit_error = False
                try:
                    bottleneck_values = [float(x) for x in bottleneck_string.split(',')]
                except ValueError:
                    print('Invalid float found, recreating bottleneck')
                    hit_error = True

                if hit_error:
                    create_bottleneck_file(bottleneck_path, image_lists, label_name, index,
                                       image_dir, category, sess, jpeg_data_tensor,
                                       bottleneck_tensor)
                with open(bottleneck_path, 'r') as bottleneck_file:
                    bottleneck_string = bottleneck_file.read()
                    bottleneck_values = [float(x) for x in bottleneck_string.split(',')]

                bottlenecks += 1
        print ('bottleneck files created {}'.format(str(bottlenecks)))

In [87]:
# add the last layer for training
def final_training_layer(class_count, final_tensor_name, bottleneck_tensor):

    with tf.name_scope('input'):
        bottleneck_input = tf.placeholder_with_default(bottleneck_tensor, shape=[None, BOTTLENECK_TENSOR_SIZE],name='BottleneckInputPlaceholder')

        ground_truth_input = tf.placeholder(tf.float32,[None, class_count],name='GroundTruthInput')
        layer_name = 'final_training_ops'
    with tf.name_scope(layer_name):
        with tf.name_scope('weights'):
            initial_value = tf.truncated_normal([BOTTLENECK_TENSOR_SIZE, class_count],
                                          stddev=0.001)
            layer_weights = tf.Variable(initial_value, name='final_weights')

        with tf.name_scope('biases'):
            layer_biases = tf.Variable(tf.zeros([class_count]), name='final_biases')
        with tf.name_scope('Wx_plus_b'):
            logits = tf.matmul(bottleneck_input, layer_weights) + layer_biases
  
    final_tensor = tf.nn.softmax(logits, name=final_tensor_name)
  
    with tf.name_scope('cross_entropy'):
        cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=ground_truth_input, logits=logits)
        with tf.name_scope('total'):
            cross_entropy_mean = tf.reduce_mean(cross_entropy)
  
    with tf.name_scope('train'):
        optimizer = tf.train.GradientDescentOptimizer(lr)
        train_step = optimizer.minimize(cross_entropy_mean)

    return (train_step, cross_entropy_mean, bottleneck_input, ground_truth_input,final_tensor)


In [88]:
# create evaluation step to test the last layer
def evaluation_step(result_tensor, ground_truth_tensor):
    with tf.name_scope('accuracy'):
        with tf.name_scope('correct_prediction'):
            prediction = tf.argmax(result_tensor, 1)
            correct_prediction = tf.equal(prediction, tf.argmax(ground_truth_tensor, 1))
        with tf.name_scope('accuracy'):
            evaluation_step = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    return evaluation_step, prediction

In [89]:
def run(train_image_dir,train_bottleneck_dir,output_graph_path):
    # Building the InceptionV3 graph
    graph, bottleneck_tensor, jpeg_data_tensor, resized_image_tensor = (create_inception_graph())
    train_image_lists = Create_Image_lists ( train_image_dir )
    
    with tf.Session(graph=graph) as sess:
        # Here we cashe bottlenecks on .txt files to use it on training
        cache_bottlenecks(sess, train_image_lists, train_image_dir, train_bottleneck_dir, jpeg_data_tensor,bottleneck_tensor)

        # Add the new layer that we'll be training.
        (train_step, cross_entropy, bottleneck_input, ground_truth_input,
        final_tensor) = final_training_layer(num_classes,final_tensor_name, bottleneck_tensor)

        # Create the operations we need to evaluate the accuracy of our new layer.
        evaluation_step, prediction = evaluation_step(final_tensor, ground_truth_input)
        #load trained bottlenecks
        train_bottlenecks,train_labels = load_bottlenecks(train_bottleneck_dir)
        init = tf.global_variables_initializer()
        sess.run(init)
        #learn the last layer for the number of steps 
        for step in range(training_steps):
            _ = sess.run([ train_step],feed_dict={bottleneck_input: train_bottlenecks,
                                     ground_truth_input: train_labels})
        
            if (step % display_step) == 0 :
                train_accuracy, cross_entropy_value = sess.run([evaluation_step, cross_entropy],
                feed_dict={bottleneck_input: train_bottlenecks,ground_truth_input: train_labels})
                print('Step %d: Train accuracy = %.1f%% , Loss = %f ' % ( step,train_accuracy * 100,
                                                                         cross_entropy_value))     
        output_graph_def = graph_util.convert_variables_to_constants(sess, graph.as_graph_def(), 
                                                                     [final_tensor_name])
        if not os.path.exists(output_graph_path):
            os.makedirs(output_graph_path)
            
        if not os.path.exists(output_graph_path+"retrained_graph.pb"):
            with gfile.FastGFile(output_graph_path+"retrained_graph.pb", 'wb') as f:
                f.write(output_graph_def.SerializeToString())
            print("Retrained graph saved successfully")

            
def test(test_image_dir,output_graph_path):
    #testing
    #load the graph
    with tf.gfile.FastGFile(output_graph_path+"retrained_graph.pb", 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        _ = tf.import_graph_def(graph_def, name='')

    #test
    print("Testing started ...... this may take a while ^_^ ")
    with tf.Session() as sess:
        correctly_classified = 0
        total_images_num=0

        for Class in os.listdir ( test_image_dir ):
            Label=get_label(Class)
            for i in range(len(Label)):
                if Label[i]==1:
                    label=i
            for img in os.listdir ( test_image_dir+Class):
                total_images_num+=1
                image_path = os.path.join ( test_image_dir+Class, img )
                image_data = tf.gfile.FastGFile ( image_path, 'rb' ).read ()
                softmax_tensor = sess.graph.get_tensor_by_name('final_result:0')
                predictions = sess.run(softmax_tensor,{'DecodeJpeg/contents:0': image_data})
                top_k = predictions[0].argsort()[-len(predictions[0]):][::-1]
                pred=top_k[0]
                if pred==label:
                    correctly_classified+=1
    test_acc=(correctly_classified/total_images_num)*100
    print('Test accuracy = %.3f%% ' % ( test_acc ))

In [92]:
# train non segmentation images
run(train_image_dir_without_segmentation,train_bottleneck_dir_without_segmentation,
    output_graph_path_without_segmentation)

./tf_files/train_bottlenecks_without_segmentation/alpine sea holly-35\image_06984.jpg
dataset/alpine sea holly-35\image_06984.jpg
create bottle  {'_graph': <tensorflow.python.framework.ops.Graph object at 0x000001C69EB045F8>, '_opened': False, '_closed': False, '_current_version': 0, '_extend_lock': <unlocked _thread.lock object at 0x000001C6AB085D28>, '_target': b'', '_delete_lock': <unlocked _thread.lock object at 0x000001C6AB085F80>, '_dead_handles': [], '_config': device_count {
  key: "CPU"
  value: 1
}
device_count {
  key: "GPU"
}
gpu_options {
  experimental {
  }
}
, '_add_shapes': False, '_session': <Swig Object of type 'TF_Session *' at 0x000001C6A936C720>, '_default_graph_context_manager': <contextlib._GeneratorContextManager object at 0x000001C6A576A7F0>, '_default_session_context_manager': <contextlib._GeneratorContextManager object at 0x000001C6A576AE80>}
Error during processing file dataset/alpine sea holly-35\image_06984.jpg
The Session graph is empty.  Add operations 

FileNotFoundError: [Errno 2] No such file or directory: './tf_files/train_bottlenecks_without_segmentation/alpine sea holly-35\\image_06984.jpg.txt'